In [ ]:
import pandas as pd
import numpy as np
from modnet.featurizers.presets import Matminer2023Featurizer
from modnet.preprocessing import nmi_target ,get_cross_nmi, get_features_relevance_redundancy, get_features_dyn
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_pickle('matbench_data_with_mpid.pkl')
df.set_index('mp-id',inplace=True)
df.drop(index=np.nan, inplace=True)
df.drop(columns=['last phdos peak', 'Composition','Spacegroup','Formula'], inplace=True)

In [ ]:
lobpy_stat= pd.read_csv('Small_basis_sum_stat.csv', sep=';', index_col='mp-ids')
lobpy_stat.sort_index(inplace=True)

In [ ]:
feat = Matminer2023Featurizer() #convinience method to add features using matminer from modnet

In [ ]:
df_comp = feat.featurize_composition(df) # get composition based features from matminer

In [ ]:
df_comp[df_comp.columns[df_comp.isnull().any()]].isnull().sum() 

In [ ]:
df_st = feat.featurize_structure(df) # get structure based features from matminer 

In [ ]:
df_merge = pd.concat([df_st, df_comp], axis=1)

In [ ]:
# find columns with max null values
df_merge[df_merge.columns[df_merge.isnull().any()]].isnull().sum()

In [ ]:
# drop columns with maximim null values
df_fil = df_merge.drop(columns=['GlobalSymmetryFeatures|crystal_system',
                      'AtomicPackingEfficiency|mean simul. packing efficiency',
                      'AtomicPackingEfficiency|mean abs simul. packing efficiency',
                      'Miedema|Miedema_deltaH_inter',
                      'Miedema|Miedema_deltaH_amor',
                      'Miedema|Miedema_deltaH_ss_min',
                      'YangSolidSolution|Yang delta'])

In [ ]:
df_fil.dropna(inplace=True) #drop nan rows

In [ ]:
# add lobsterpy summary stat data to dataframe
for feat in lobpy_stat.columns:
    for row, col in df_fil.iterrows():
        #print(row)
        df_fil.loc[row,feat] = lobpy_stat.loc[row,feat]
    

In [ ]:
#features = df_merge.drop(columns=['last phdos peak','Input Data|last phdos peak'])

In [ ]:
df = pd.read_pickle('matbench_data_with_mpid.pkl')
df.set_index('mp-id',inplace=True)
df.drop(index=np.nan, inplace=True)
df.drop(columns=['structure', 'Composition','Spacegroup','Formula'], inplace=True)

In [ ]:
#targets = df.filter(items = list(df_fil.index) , axis=0)

In [ ]:
#target_nmi= nmi_target(task_type='regression',df_feat=df_fil, df_target=targets,drop_constant_features=True)

In [ ]:
#cross_nmi= get_cross_nmi(df_feat=df_fil, n_jobs=14, drop_thr=0.2)

In [ ]:
for row, col in df_fil.iterrows():
    df_fil.loc[row,'last_phdos_peak']=targets.loc[row,'last phdos peak']

In [ ]:
df_fil.to_pickle('dataforml.pkl') # save cleaned data ready for ML training